### CAPM Beta and Volatility Estimation 
Author : Osho Sharma 


This file includes code to find yearly CAPM beta, along with systematic and idiosyncratic risk using monthly stock return data from CRSP (1996-2023)

In [89]:
#import libraries 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
# import data 
data = pd.read_csv('./MSF_1996_2023.csv')

#### Data Cleaning and Wrangling

In [99]:
data_filtered = data.filter(items=['date', 'PERMCO', 'vwretd', 'RET', 'SICCD',])
data_filtered.rename(columns = {'vwretd': 'market_return','RET': 'firm_return', 'TICKER' : 'ticker', 'SICCD' : 'industry_code', 'PERMCO': 'company_id'}, inplace = True)


# convert all industry code to numeric and handle errors.
data_filtered['industr_code_numeric'] = pd.to_numeric(data_filtered['industry_code'], errors='coerce')  
data_filtered = data_filtered.dropna(subset=['industr_code_numeric'])
data_filtered['industry_code'] = data_filtered['industry_code'].astype(int)
data_filtered.drop(columns=['industr_code_numeric'], inplace=True)

# Industry ranges SIC code 
industry_ranges = { 
    'agriculture': (1,999), 
    'mining': (1000,1499),
    'construction': (1500,1799),
    'manufacturing': (2000,3999),
    'transportation': (4000,4999),
    'wholesale_trade': (5000,5199),
    'retail_trade': (5200,5999),
    'finance': (6000,6799),
    'services': (7000,8999),
    'public_admin': (9000,9999)
}

# add new column for industry_name using industry_ranges
data_filtered['industry_name'] = data_filtered['industry_code'].apply(lambda x: next((k for k, v in industry_ranges.items() if v[0] <= x <= v[1]), None))
data_filtered['year'] = data_filtered['date'].str[:4]



# Filter ind_data to include monthly rows for 10 unique companies per industry_name for each year
sampled_data = data_filtered.groupby(['industry_name', 'year']).apply(
    lambda group: group[group['company_id'].isin(group.drop_duplicates('company_id').head(10)['company_id'])]
).reset_index(drop=True)

# convert all firm return & market return to float and handle errors.
sampled_data['firm_return_float'] = pd.to_numeric(sampled_data['firm_return'], errors='coerce')  
sampled_data = sampled_data.dropna(subset=['firm_return_float'])
sampled_data['firm_return'] = sampled_data['firm_return'].astype(float)
sampled_data.drop(columns=['firm_return_float'], inplace=True)
sampled_data['market_return'] = sampled_data['market_return'].astype(float)


/var/folders/qq/4_7zbq5x7dx8hhvz65nzsm740000gn/T/ipykernel_5658/1332558991.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_data = data_filtered.groupby(['industry_name', 'year']).apply(


In [92]:
sampled_data.head(24)

,date,company_id,market_return,firm_return,industry_code,industry_name,year
0,1996-01-31,9437,0.028121,0.109577,170,agriculture,1996
1,1996-02-29,9437,0.016353,0.044586,170,agriculture,1996
2,1996-03-29,9437,0.010914,0.067073,170,agriculture,1996
3,1996-04-30,9437,0.025560,0.214629,170,agriculture,1996
4,1996-05-31,9437,0.026810,0.056604,170,agriculture,1996
5,1996-06-28,9437,-0.008289,0.071429,170,agriculture,1996
6,1996-07-31,9437,-0.053851,-0.064000,170,agriculture,1996
7,1996-08-30,9437,0.032451,0.285714,170,agriculture,1996
8,1996-09-30,9437,0.052985,-0.055556,170,agriculture,1996
9,1996-10-31,9437,0.013673,0.132353,170,agriculture,1996


#### Linear Regression for Beta Estimation

In [124]:
def calculate_beta_for_window(sub_df, window, results):
    for year in sub_df['year'].unique():
        # Filter the data for the current year
        df_year = sub_df[sub_df['year'] == year]

        # For each row in the current year, get the last 'window' months of data
        for index, row in df_year.iterrows():
            df_last_n_months = sub_df[(sub_df['date'] <= row['date'])].tail(window)

            if len(df_last_n_months) == window:  # Ensure we have the required window of data
                # Market returns (X) and stock returns (Y)
                X = df_last_n_months[['excess_market_return']]
                y = df_last_n_months['excess_firm_return']

                # Fit linear regression
                model = LinearRegression()
                model.fit(X, y)
                y_actual = row['excess_firm_return']
                y_pred = model.predict([[row['excess_market_return']]])[0]
                residual = y_actual - y_pred    

                # Get beta (the coefficient of market returns)
                beta = model.coef_[0]
                
                # Append the result for this company, year, and window
                results.append({
                    'company_id': row['company_id'],
                    'year': row['year'],
                    'date': row['date'],
                    'window': window,  # Store the window size (12, 24, or 36)
                    'beta': beta,
                    'residual': residual
                })

In [ ]:
# suppress unnecessary warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# calculate excess returns.
rf_rate_monthly = 0.0425/12
sampled_data['excess_firm_return'] = sampled_data['firm_return'] - rf_rate_monthly
sampled_data['excess_market_return'] = sampled_data['market_return'] - rf_rate_monthly

results = []

# Calculate beta for 12, 24, and 36 month windows for each security - using linear regression.
windows = [12, 24, 36]
for window in windows:
    sampled_data.groupby('company_id').apply(lambda group: calculate_beta_for_window(group, window, results))


beta_data = pd.DataFrame(results)



In [128]:
beta_data.head(24)

,company_id,year,date,window,beta,residual
0,37,1996,1996-12-31,12,0.298438,0.061831
1,37,1997,1997-01-31,12,0.102284,-0.047948
2,37,1997,1997-02-28,12,0.053202,0.034905
3,37,1997,1997-03-31,12,-0.017706,0.008890
4,37,1997,1997-04-30,12,-0.027904,-0.007757
5,37,1997,1997-05-30,12,0.093821,0.028586
6,37,1997,1997-06-30,12,0.316679,0.091888
7,37,1997,1997-07-31,12,0.114469,0.023564
8,37,1997,1997-08-29,12,-0.440934,0.119427
9,37,1997,1997-09-30,12,-0.443052,0.030304
